In [ ]:
import csv
import requests
from bs4 import BeautifulSoup
from csv import DictWriter

# Step 1: Read URLs from CSV file
keyword_site_urls = dict()
with open('D:\\ThinkByte_project\Medibuddy_project\data\med_urls.csv', 'r') as csvfile:
    csvreader = csv.DictReader(csvfile)
    for row in csvreader:
        keyword = row['source3']  # 'source3' is the column containing URLs
        keyword_site_urls[keyword] = {'source3': row['source3']}  # Adjust as per your CSV structure

print("CSV data",keyword_site_urls)

apollo_responses = dict()

# Step 2: Fetch data from each URL
for keyword in keyword_site_urls:
    url = keyword_site_urls[keyword]['source3']
    response = requests.get(url)
    if response.status_code == 200:
        apollo_responses[keyword] = response.text
        print(f"HTML content fetched for {keyword}.")
    else:
        print(f"Failed to fetch webpage for {keyword}. Status code: {response.status_code}")
print("responce",apollo_responses)
# Step 3: Process fetched data
apollo_data = []
for keyword in apollo_responses:
    soup = BeautifulSoup(apollo_responses[keyword], 'html.parser')
    # Your data processing logic here

    drug_title = soup.find_all('div', class_=lambda value: value and value.startswith('PdpImagePlaceholder_title'))[0].text.strip()
    prescription_drug = soup.find_all('div', class_=lambda value: value and value.startswith('PdpImagePlaceholder_pdpTagWrapper'))[0].get_text()
    product_info_div = soup.find('div', class_=lambda value: value and value.startswith('PdpImagePlaceholder_productInfoRoot'))
    product_info_grids = soup.find_all('div', class_=lambda value: value and value.startswith('Grid_Item'))
    manufacturer = product_info_grids[1].get_text()
    
    product_data = soup.find_all('div', class_='yh')
    product_info_blocks = product_data[0].find_all('div', class_='RP')
    about = ''
    uses = ''
    benefits = ''
    directions = ''
    storage = ''
    side_effects = ''
    for pi_block in product_info_blocks:
        title = pi_block.find('h2').get_text()
        desc = '\n'.join(para.get_text() for para in pi_block.find_all('p'))
        if not desc:
            desc = '\n'.join(para.get_text() for para in pi_block.find_all('li'))
        if not desc:
            desc = pi_block.find('div', class_='VP').get_text()
        if 'About' in title:
            about = desc
        elif 'Uses' in title:
            uses = desc
        elif 'Benefits' in title:
            benefits = desc
        elif 'Directions' in title:
            directions = desc
        elif 'Storage' in title:
            storage = desc
        elif 'Side Effects' in title:
            side_effects = desc
    
    try:
        in_depth_info_blocks = product_data[1].find_all('div', class_='RP')
        warnings = ''
        interactions_checkers = ''
        interactions = ''
        habit = ''
        diet = ''
        advise = ''
        for pi_block in in_depth_info_blocks:
            title = pi_block.find('h2').get_text()
            desc = '\n'.join(para.get_text() for para in pi_block.find_all('p'))
            if not desc:
                desc = '\n'.join(para.get_text() for para in pi_block.find_all('li'))
            if not desc:
                desc = pi_block.find('div', class_='VP').get_text()
            if 'Warnings' in title:
                warnings = desc
            elif 'Checker' in title:
                interactions_checkers = desc
            elif 'Interactions' in title:
                interactions = desc
            elif 'Habit' in title:
                habit = desc
            elif 'Diet' in title:
                diet = desc
            elif 'Advise' in title:
                advise = desc
                
    except:
        in_depth_info = ''
    try:
        patients_concern = product_data[2].get_text()
    except:
        patients_concern = ''

    try:
        safety_text = ''
        safety_data = soup.find('div', class_='bb').find_all('div', class_="W_")
        for row in safety_data:
            safety_title = row.find('p', class_='Ld').get_text()
            safety_tag = row.find('p', class_='_b').get_text()
            safety_detail = row.find_all('p', class_='Ld')[1].get_text()
            safety_text += 'Title:' + safety_title + '\n' + 'Tag:' + safety_tag + '\n' + 'Detail:' + safety_detail + '\n\n'
        
        print (safety_text)
    except :
        safety_text = ''
    
    faqs_text = ''
    faqs_div = soup.find_all('div', class_='a')
    for faq_div in faqs_div:
        ques = faq_div.find('div', class_='b').get_text()
        ans = faq_div.find('div', class_='g').get_text()
        faqs_text += ques + '\n' + ans + '\n\n'
    
    manufacturer_details = soup.find_all('div', class_='pJ')

    apollo_row = {
        # 'images': images,
        'drug_title': drug_title,
        'prescription_drug': prescription_drug,
        'product_info_div': product_info_div,
        'product_info_grids': product_info_grids,
        'manufacturer': manufacturer,
        'about': about,
        'uses': uses,
        'benefits': benefits,
        'directions': directions,
        'storage': storage,
        'side_effects': side_effects,
        'warnings': warnings,
        'interactions': interactions,
        'interactions_checkers': interactions_checkers,
        'habit': habit,
        'diet': diet,
        'advise': advise,
        'patients_concern': patients_concern,
        'safety_text': safety_text,
        'faqs_text': faqs_text,
        'manufacturer_details': manufacturer_details
        
    }
    apollo_data.append(apollo_row)
    apollo_dict[keyword] = apollo_row
    # break

# Step 4: Save processed data into a new CSV file
with open('apollo_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['drug_title', 'prescription_drug', 'product_info_div','product_info_grids','manufacturer','about','uses','benefits','directions','storage','side_effects','warnings','interactions','interactions_checkers','habit','diet','advise','patients_concern','safety_text','faqs_text','manufacturer_details']  # Update with your field names
    writer = DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(apollo_data)

print("Data saved to apollo_data.csv.")


In [ ]:
import csv
import requests
from bs4 import BeautifulSoup
from csv import DictWriter

# Step 1: Read URLs from CSV file
keyword_site_urls = dict()
with open('D:\\ThinkByte_project\Medibuddy_project\data\med_urls.csv', 'r') as csvfile:
    csvreader = csv.DictReader(csvfile)
    for row in csvreader:
        keyword = row['source3'][1]  # 'source3' is the column containing URLs
        keyword_site_urls[keyword] = {'source3': row['source3']}  # Adjust as per your CSV structure

apollo_responses = dict()

# Step 2: Fetch data from each URL
for keyword in keyword_site_urls:
    url = keyword_site_urls[keyword]['source3']
    response = requests.get(url)
    if response.status_code == 200:
        apollo_responses[keyword] = response.text
        print(f"HTML content fetched for {keyword}.")
    else:
        print(f"Failed to fetch webpage for {keyword}. Status code: {response.status_code}")

# Step 3: Process fetched data
apollo_data = []

for keyword in apollo_responses:
    try:
        soup = BeautifulSoup(apollo_responses[keyword], 'html.parser')
        drug_title = soup.find_all('div', class_=lambda value: value and value.startswith('PdpImagePlaceholder_title'))[0].get_text()
        prescription_drug = soup.find_all('div', class_=lambda value: value and value.startswith('PdpImagePlaceholder_pdpTagWrapper'))[0].get_text()
        product_info_div = soup.find('div', class_=lambda value: value and value.startswith('PdpImagePlaceholder_productInfoRoot'))
        product_info_grids = soup.find_all('div', class_=lambda value: value and value.startswith('Grid_Item'))
        manufacturer = product_info_grids[1].get_text()
        
        product_data = soup.find_all('div', class_='yh')
        print(product_data)
        ##############################
        try:
            product_info_blocks = product_data[0].find_all('div', class_='RP')
            about = ''
            uses = ''
            benefits = ''
            directions = ''
            storage = ''
            side_effects = ''
            for pi_block in product_info_blocks:
                title = pi_block.find('h2').get_text()
                desc = '\n'.join(para.get_text() for para in pi_block.find_all('p'))
                if not desc:
                    desc = '\n'.join(para.get_text() for para in pi_block.find_all('li'))
                if not desc:
                    desc = pi_block.find('div', class_='VP').get_text()
                if 'About' in title:
                    about = desc
                elif 'Uses' in title:
                    uses = desc
                elif 'Benefits' in title:
                    benefits = desc
                elif 'Directions' in title:
                    directions = desc
                elif 'Storage' in title:
                    storage = desc
                elif 'Side Effects' in title:
                    side_effects = desc
        except Exception as e:
            print(e)  
        ####################################  
        try:
            in_depth_info_blocks = product_data[1].find_all('div', class_='RP')
            warnings = ''
            interactions_checkers = ''
            interactions = ''
            habit = ''
            diet = ''
            advise = ''
            for pi_block in in_depth_info_blocks:
                title = pi_block.find('h2').get_text()
                desc = '\n'.join(para.get_text() for para in pi_block.find_all('p'))
                if not desc:
                    desc = '\n'.join(para.get_text() for para in pi_block.find_all('li'))
                if not desc:
                    desc = pi_block.find('div', class_='VP').get_text()
                if 'Warnings' in title:
                    warnings = desc
                elif 'Checker' in title:
                    interactions_checkers = desc
                elif 'Interactions' in title:
                    interactions = desc
                elif 'Habit' in title:
                    habit = desc
                elif 'Diet' in title:
                    diet = desc
                elif 'Advise' in title:
                    advise = desc
                    
        except:
            in_depth_info = ''
        try:
            patients_concern = product_data[2].get_text()
        except:
            patients_concern = ''

        try:
            safety_text = ''
            safety_data = soup.find('div', class_='bb').find_all('div', class_="W_")
            for row in safety_data:
                safety_title = row.find('p', class_='Ld').get_text()
                safety_tag = row.find('p', class_='_b').get_text()
                safety_detail = row.find_all('p', class_='Ld')[1].get_text()
                safety_text += 'Title:' + safety_title + '\n' + 'Tag:' + safety_tag + '\n' + 'Detail:' + safety_detail + '\n\n'
            
            print (safety_text)
        except :
            safety_text = ''
        
        faqs_text = ''
        faqs_div = soup.find_all('div', class_='a')
        for faq_div in faqs_div:
            ques = faq_div.find('div', class_='b').get_text()
            ans = faq_div.find('div', class_='g').get_text()
            faqs_text += ques + '\n' + ans + '\n\n'
        
        manufacturer_details = soup.find_all('div', class_='pJ')

        apollo_row = {
         
            'drug_title': drug_title,
            'prescription_drug': prescription_drug,
            'product_info_div': product_info_div,
            'product_info_grids': product_info_grids,
            'manufacturer': manufacturer,
            'about': about,
            'uses': uses,
            'benefits': benefits,
            'directions': directions,
            'storage': storage,
            'side_effects': side_effects,
            'warnings': warnings,
            'interactions': interactions,
            'interactions_checkers': interactions_checkers,
            'habit': habit,
            'diet': diet,
            'advise': advise,
            'patients_concern': patients_concern,
            'safety_text': safety_text,
            'faqs_text': faqs_text,
            'manufacturer_details': manufacturer_details
            
        }
        apollo_data.append(apollo_row)
        
        
    except Exception as e:
        print (e)

# Step 4: Save processed data into a new CSV file
f = open('apollo.csv', 'w')
writer = DictWriter(f, fieldnames = apollo_data[0].keys())
writer.writeheader()
writer.writerows(apollo_data)

print("Data saved to netmeds_data.csv.")


In [ ]:
import csv
import requests
from bs4 import BeautifulSoup
from csv import DictWriter

# Step 1: Read URLs from CSV file
keyword_site_urls = dict()
with open('D:\\ThinkByte_project\Medibuddy_project\data\med_urls.csv', 'r') as csvfile:
    csvreader = csv.DictReader(csvfile)
    for row in csvreader:
        keyword = row['source3']  # 'source3' is the column containing URLs
        keyword_site_urls[keyword] = {'source3': row['source3']}  # Adjust as per your CSV structure

apollo_responses = dict()
print(keyword_site_urls)

# Step 2: Fetch data from each URL
for keyword in keyword_site_urls:
    url = keyword_site_urls[keyword]['source3']
    response = requests.get(url)
    if response.status_code == 200:
        apollo_responses[keyword] = response.text
        print(f"HTML content fetched for {keyword}.")
    else:
        print(f"Failed to fetch webpage for {keyword}. Status code: {response.status_code}")

# Step 3: Process fetched data
apollo_data = []

for keyword in apollo_responses:
    try:
        soup = BeautifulSoup(apollo_responses[keyword], 'html.parser')
        drug_title = soup.find('div', class_='PdpImagePlaceholder_title__kN_jC').find('div', class_='OJ').find('h1').get_text()
        
        prescription_drug = soup.find_all('div', class_=lambda value: value and value.startswith('PdpImagePlaceholder_pdpTagWrapper'))[0].get_text()
        product_info_div = soup.find('div', class_=lambda value: value and value.startswith('PdpImagePlaceholder_productInfoRoot'))
        product_info_grids = soup.find_all('div', class_=lambda value: value and value.startswith('Grid_Item'))
        manufacturer = product_info_grids[1].get_text()
        
        product_data = soup.find_all('div', class_='yh')
        
        print(drug_title)
        
    except Exception as e:
        print (e)



In [2]:
import requests
from bs4 import BeautifulSoup
url = "https://www.apollopharmacy.in/medicine/benadryl-cough-formula-syrup-450ml"

r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
for i in soup.find_all("code"):
    print(i.text)

In [5]:
drug_title = drug_title = soup.find_all('div', class_=lambda value: value and value.startswith('PdpImagePlaceholder_title'))[0].text.strip()
print(drug_title)

AttributeError: 'NoneType' object has no attribute 'text'